In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm

In [ ]:
class Traffic:
    '''Class for the traffic problem'''
    def __init__(self,tau,N,L,v_m,rho_m,tMax,f0,method='ftcs'):
            
        self.v_m=v_m
        self.rho_m=rho_m
        self.tMax=tMax
        self.method=method
        
        # Coefficient
        h=L/N
        coeff=tau/(2.0*h)
    
        # Initialize rho
        self.xlin=np.linspace(0,L,N)
        self.rho=np.zeros((tMax,N))
        self.rho[0,:]=f0(self.xlin,rho_m,L)
        self.rho[0,-1]=self.rho[0,0] # Make sure PBCs are enforced
    
        # Need this line to get us started!
        self.rho[0,int(N/2)]=0.5*rho_m
        
        # solve rho(x,t)
        self.rho=self.solve_traffic(coeff,self.rho,self.F,self.c)
        
        # Get grids, helpful for plotting
        self.tlin=np.linspace(0,tMax,tMax)*tau
        self.x,self.t=np.meshgrid(self.xlin,self.tlin)        

    def solve_traffic(self,coeff,rho,F,c):
        '''Function to solve advection/Burgers equation'''
        # Propagate advection equation
        for n in range(self.tMax-1):
            
            # To make equations more clear, define rho^n's here:
            i=rho[n,1:N-1]
            im=rho[n,0:N-2]
            ip=rho[n,2:N]

            if self.method=='ftcs':
                rho[n+1,1:N-1] = i-coeff*(F(ip)-F(im))
                # Apply periodic boundary conditions
                rho[n+1,0] = im[0]-coeff*(F(i[0])-F(ip[-1]))
                rho[n+1,N-1] = ip[-1]-coeff*(F(im[0])-F(i[-1]))

            elif self.method=='lax':
                rho[n+1,1:N-1] = 0.5*(ip+im)-coeff*(F(ip)-F(im))
                rho[n+1,0] = 0.5*(i[0]+ip[-1])-coeff*(F(i[0])-F(ip[-1]))
                rho[n+1,N-1] = 0.5*(im[0]+i[-1])-coeff*(F(im[0])-F(i[-1]))                        
 
            elif self.method=='lw':
                rho[n+1,1:N-1]= i-coeff*(F(ip)-F(im))+2.*coeff**2*( c(0.5*(ip+i)) * \
                                (F(ip)-F(i))-c(0.5*(im+i))*(F(i)-F(im)) )
                
                rho[n+1,0]= im[0]-coeff*(F(i[0])-F(ip[-1]))+2.*coeff**2*( c(0.5*(i[0]+im[0])) * \
                                (F(i[0])-F(im[0]))-c(0.5*(ip[-1]+im[0]))*(F(im[0])-F(ip[-1])) )
                
                rho[n+1,N-1]= ip[-1]-coeff*(F(im[0])-F(i[-1]))+2.*coeff**2*( c(0.5*(im[0]+ip[-1])) * \
                                (F(im[0])-F(ip[-1]))-c(0.5*(i[-1]+ip[-1]))*(F(ip[-1])-F(i[-1])) )
        
        return rho

    def F(self,rho):
        '''Function to calculate flow'''
        return rho*self.v_m*(1.-rho/self.rho_m)
    
    def c(self,rho):
        '''Function to calculate flow'''
        return self.v_m*(1.-2.*rho/self.rho_m)

# End of class-------------
    
    
def square_pulse(x,rho_m,L):
    '''Function finite line of cars ar stoplight'''
    N=len(x)
    rho=np.zeros(N)
    rho[int(N/4):int(N/2)]=rho_m
    
    return rho 

In [ ]:
L=400.
N=80
tau=0.2
v_m=25.
rho_m=1.
tMax=20

f0=square_pulse

tr_ftcs=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='ftcs')
tr_lax=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='lax')
tr_lw=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='lw')

# Plots
fig = plt.figure(figsize=(15,15))
ax0 = fig.add_subplot(3, 2, 1, projection='3d')
ax1 = fig.add_subplot(3, 2, 2,)
ax2 = fig.add_subplot(3, 2, 3, projection='3d')
ax3 = fig.add_subplot(3, 2, 4,)
ax4 = fig.add_subplot(3, 2, 5, projection='3d')
ax5 = fig.add_subplot(3, 2, 6,)
axes=[ax0,ax1,ax2,ax3,ax4,ax5]

# 3D plots:
mycmap=cm.plasma
ax0.plot_surface(tr_ftcs.x,tr_ftcs.t,tr_ftcs.rho,cmap=mycmap)
ax2.plot_surface(tr_lax.x,tr_lax.t,tr_lax.rho,cmap=mycmap)
ax4.plot_surface(tr_lw.x,tr_lw.t,tr_lw.rho,cmap=mycmap)

# Contour plots
CS_ftcs=ax1.contour(tr_ftcs.x,tr_ftcs.t,tr_ftcs.rho)
CS_lax=ax3.contour(tr_lax.x,tr_lax.t,tr_lax.rho)
CS_lw=ax5.contour(tr_lw.x,tr_lw.t,tr_lw.rho)
ax1.clabel(CS_ftcs)
ax3.clabel(CS_lax)
ax5.clabel(CS_lw)

# Label axes
titles=['FTCS','FTCS','Lax','Lax','Lax-Wendroff','Lax-Wendroff']
for axis in range(6):
    axes[axis].set_xlabel('position')
    axes[axis].set_ylabel('time')
    axes[axis].set_title(titles[axis])
    if axis%2 == 0: axes[axis].set_zlabel(r'$\rho$')

plt.savefig('traffic.pdf',bbox_inches='tight')
plt.show()

In [ ]:
# Run for 100 time steps
L=400.
N=80
tau=0.2
v_m=25.
rho_m=1.
tMax=100

f0=square_pulse

tr_ftcs=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='ftcs')
tr_lax=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='lax')
tr_lw=Traffic(tau,N,L,v_m,rho_m,tMax,f0,method='lw')

# Plots
fig = plt.figure(figsize=(15,15))
ax0 = fig.add_subplot(3, 2, 1, projection='3d')
ax1 = fig.add_subplot(3, 2, 2,)
ax2 = fig.add_subplot(3, 2, 3, projection='3d')
ax3 = fig.add_subplot(3, 2, 4,)
ax4 = fig.add_subplot(3, 2, 5, projection='3d')
ax5 = fig.add_subplot(3, 2, 6,)
axes=[ax0,ax1,ax2,ax3,ax4,ax5]

# 3D plots:
mycmap=cm.plasma
ax0.plot_surface(tr_ftcs.x,tr_ftcs.t,tr_ftcs.rho,cmap=mycmap)
ax2.plot_surface(tr_lax.x,tr_lax.t,tr_lax.rho,cmap=mycmap)
ax4.plot_surface(tr_lw.x,tr_lw.t,tr_lw.rho,cmap=mycmap)

# Contour plots
CS_ftcs=ax1.contour(tr_ftcs.x,tr_ftcs.t,tr_ftcs.rho)
CS_lax=ax3.contour(tr_lax.x,tr_lax.t,tr_lax.rho)
CS_lw=ax5.contour(tr_lw.x,tr_lw.t,tr_lw.rho)
ax1.clabel(CS_ftcs)
ax3.clabel(CS_lax)
ax5.clabel(CS_lw)

# Label axes
titles=['FTCS','FTCS','Lax','Lax','Lax-Wendroff','Lax-Wendroff']
for axis in range(6):
    axes[axis].set_xlabel('position')
    axes[axis].set_ylabel('time')
    axes[axis].set_title(titles[axis])
    if axis%2 == 0: axes[axis].set_zlabel(r'$\rho$')

plt.savefig('traffic_100.pdf',bbox_inches='tight')
plt.show()